In [ ]:
import cv2
img_path = "../../Dataset/isic/HAM10000/images/ISIC_0024748.jpg"

img = cv2.imread(img_path)[:, :, ::-1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_diagonal_cross_kernel(size=13):
    kernel = np.zeros((size, size), dtype=np.uint8)
    center = size // 2

    for i in range(0, size, 2):
        kernel[center, i] = 1
        kernel[i, center] = 1

    for i in range(0, size, 2):
        kernel[i, i] = 1
        kernel[i, size - 1 - i] = 1
    
    kernel[center, center - 1:center + 2] = 0
    kernel[center - 1:center + 2, center] = 0
    kernel[center - 1, center - 1] = 0
    kernel[center - 1, center + 1] = 0
    kernel[center + 1, center - 1] = 0
    kernel[center + 1, center + 1] = 0

    return kernel

kernel_struct = generate_diagonal_cross_kernel(13)

plt.imshow(kernel_struct, cmap="gray")
plt.title("Generated 13×13 Diagonal Cross Kernel with Gaps")
plt.axis("off")
plt.savefig("./kernel.png", dpi=400, bbox_inches="tight")
plt.show()

print(kernel_struct)


In [ ]:
skin_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
kernel_struct = generate_diagonal_cross_kernel(27)

top_hat = cv2.morphologyEx(skin_image, cv2.MORPH_BLACKHAT, kernel_struct)
_, binary_mask = cv2.threshold(top_hat, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

kernel_open = cv2.getStructuringElement(cv2.MORPH_CROSS, (9,9))
binary_mask_cleaned = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel_open)
img = cv2.imread(img_path)
image_inpainted = cv2.inpaint(img, binary_mask_cleaned, inpaintRadius=5, flags=cv2.INPAINT_TELEA)

fig, axes = plt.subplots(1, 4, figsize=(16, 6))
titles = ["Original Image", "Black-hat (Extracted Hair)", "Otsu Binary Mask", "Inpainted"]
images = [img, top_hat, binary_mask_cleaned, image_inpainted]

for ax, img, title in zip(axes, images, titles):
    if len(img.shape) == 2:
        ax.imshow(img, cmap="gray")
    else:
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax.set_title(title)
    ax.axis("off")

plt.tight_layout()
plt.savefig("./aug_show_flow.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
from tqdm import tqdm
import os
folder = ["train", "test", "val"]
types = ['MEL', 'AKIEC', 'BCC', 'NV', 'BKL', 'VASC', 'DF']

for i in folder:
    for j in types:
        for k in tqdm(os.listdir("/home/hank52052/Dataset/isic/HAM10000/yolo_format_process/{}/{}/".format(i,j))):
            img_path = "/home/hank52052/Dataset/isic/HAM10000/yolo_format_process/{}/{}/".format(i,j) + k
        
            skin_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            kernel_struct = generate_diagonal_cross_kernel(27)
            
            top_hat = cv2.morphologyEx(skin_image, cv2.MORPH_BLACKHAT, kernel_struct)
            _, binary_mask = cv2.threshold(top_hat, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            
            kernel_open = cv2.getStructuringElement(cv2.MORPH_CROSS, (9,9))
            binary_mask_cleaned = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel_open)
            
            img = cv2.imread(img_path)[:, :, ::-1]
            image_inpainted = cv2.inpaint(img, binary_mask_cleaned, inpaintRadius=5, flags=cv2.INPAINT_TELEA)
            plt.imsave(img_path, image_inpainted)

In [ ]:
plt.subplot(121)
plt.imshow(img)
plt.subplot(122)
plt.imshow(image_inpainted)
plt.show()

In [ ]:

import os
import shutil

base_dir = "/home/hank52052/Dataset/isic/HAM10000/cheat_four"

categories = {
    "MEL": "MEL",
    "BCC": "BCC",
    "AKIEC": "SCC",
    "BKL": "Benign",
    "DF": "Benign",
    "NV": "Benign",
    "VASC": "Benign",
}

for split in ["train", "val", "test"]:
    split_path = os.path.join(base_dir, split)
    for old_cat in os.listdir(split_path):
        old_cat_path = os.path.join(split_path, old_cat)
        if os.path.isdir(old_cat_path):
            new_cat = categories.get(old_cat, "Benign")
            new_cat_path = os.path.join(split_path, new_cat)
            if new_cat != old_cat:
                os.makedirs(new_cat_path, exist_ok=True)
                for file in os.listdir(old_cat_path):
                    old_file_path = os.path.join(old_cat_path, file)
                    new_file_path = os.path.join(new_cat_path, file)
                    shutil.move(old_file_path, new_file_path)
                os.rmdir(old_cat_path)

print("資料分類完成，原目錄已更新。")


In [ ]:
pip install thop fvcore

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO("./yolo_training/962/weights/best.pt")

In [ ]:
total_params = sum(p.numel() for p in model.model.parameters())

In [ ]:
total_params

In [ ]:
import cv2
cv2.imread("../../Dataset/isic/HAM10000/cheat_four/train/BCC/ISIC_0024345.jpg").shape

In [ ]:
from fvcore.nn import FlopCountAnalysis
import torch

model_c = model.model.eval()
dummy = torch.randn(1, 3, 450, 600)

flops = FlopCountAnalysis(model_c, dummy)
print(f"Total FLOPs: {flops.total() / 1e9:.3f} GFLOPs")

In [ ]:
import torch
import time

model_c = model.model.eval().cuda()

dummy = torch.randn(1, 3, 450, 600).cuda()

for _ in range(10):
    _ = model_c(dummy)

torch.cuda.synchronize()
start = time.time()

N = 100 
for _ in range(N):
    _ = model_c(dummy)

torch.cuda.synchronize()
end = time.time()

avg_ms = (end - start) / N * 1000
fps = 1000 / avg_ms

print(f"Inference Time: {avg_ms:.3f} ms")
print(f"FPS: {fps:.2f}")


In [ ]:
import torch
import time

model_c = model.model.eval().cpu()    


dummy = torch.randn(1, 3, 450, 600).cpu()

for _ in range(3):
    _ = model_c(dummy)

start = time.time()

N = 20   
for _ in range(N):
    _ = model_c(dummy)

end = time.time()

avg_ms = (end - start) / N * 1000
fps = 1000 / avg_ms

print(f"[CPU] Inference Time: {avg_ms:.2f} ms")
print(f"[CPU] FPS: {fps:.2f}")


In [ ]:
import pandas as pd
import cv2
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
import tensorflow as tf
import keras_cv_attention_models as cv
tf.test.is_gpu_available()
import matplotlib.pyplot as plt
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import sys
from keras_flops.flops_counter import get_flops

def isic_model(shape=(450, 600, 3)):
    model = tf.keras.Sequential()
    model.add(cv.mobilevit.MobileViT_V2_050(num_classes=0, input_shape=(450, 600, 3)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(4))
    model.add(tf.keras.layers.Softmax())
    return model

tf.test.is_gpu_available()

In [ ]:
import numpy as np
import time

model = isic_model()

input_shape = (1, 450, 600, 3)
dummy = np.random.random(input_shape).astype(np.float32)


total_params = model.count_params()
print(f"Total Params: {total_params/1e6:.3f} M")

def get_flops_tf2(model, input_shape=(1, 450, 600, 3)):
    print("--- Starting FLOPs calculation (TF 2.x method) ---")
    
   
    concrete_func = tf.function(model).get_concrete_function(
        tf.TensorSpec(input_shape, tf.float32)
    )

    frozen_output = convert_variables_to_constants_v2(
        concrete_func,
        lower_control_flow=False
    )
    
    if isinstance(frozen_output, (list, tuple)) and len(frozen_output) == 2:
        graph_def = frozen_output[1]
    elif hasattr(frozen_output, 'graph') and hasattr(frozen_output.graph, 'as_graph_def'):
        graph_def = frozen_output.graph.as_graph_def()
    else:
        raise ValueError(f"Unexpected return type from convert_variables_to_constants_v2: {type(frozen_output)}")


    flops = tf.compat.v1.profiler.profile(
        graph=graph_def,
        options=tf.compat.v1.profiler.ProfileOptionBuilder.float_operation(),
    )
    
    return flops.total_float_ops
try:
    flops = get_flops_tf2(model, input_shape=(1,450,600,3))
    print(f"Total FLOPs: {flops/1e9:.3f} GFLOPs")
except Exception as e:
    print("FLOPs calculation failed, reason:", e)



@tf.function
def inference_step(x):
    return model(x, training=False)

def measure_inference(device, dummy, runs=50):
    with tf.device(device):
        # warm-up
        for _ in range(10):
            _ = inference_step(dummy)

        start = time.time()
        for _ in range(runs):
            _ = inference_step(dummy)
        end = time.time()

        avg_ms = (end - start) / runs * 1000
        fps = 1000 / avg_ms
    return avg_ms, fps


try:
    gpu_time, gpu_fps = measure_inference("/GPU:0", dummy)
    print(f"[GPU] Inference Time: {gpu_time:.3f} ms   | FPS: {gpu_fps:.2f}")
except:
    print("[GPU] No GPU available")


cpu_time, cpu_fps = measure_inference("/CPU:0", dummy, runs=20)
print(f"[CPU] Inference Time: {cpu_time:.3f} ms   | FPS: {cpu_fps:.2f}")


In [ ]:
import pandas as pd
import cv2
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
import tensorflow as tf
import keras_cv_attention_models as cv 
import time

try:
    from keras_flops import get_flops
except ImportError:
    try:
        from keras_flops.flops_counter import get_flops
    except ImportError:
        print("Error: The 'keras_flops' package is not installed or the import path is incorrect.")
        get_flops = None


os.environ["TF_USE_LEGACY_KERAS"] = "1"

def isic_model(shape=(450, 600, 3)):
    model = tf.keras.Sequential()
    model.add(cv.tinyvit.TinyViT_5M(num_classes=0, input_shape=shape, pretrained=None))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(4))
    model.add(tf.keras.layers.Softmax())
    return model

INPUT_SIZE = (450, 600, 3)
input_shape = (1,) + INPUT_SIZE
model = isic_model(shape=INPUT_SIZE) 
dummy = np.random.random(input_shape).astype(np.float32)



@tf.function
def inference_step(x):
    return model(x, training=False)

def measure_inference(device, dummy, runs=50):
    """測量指定設備上的平均推理時間和 FPS"""
    with tf.device(device):
        for _ in range(10):
            _ = inference_step(dummy)

        start = time.time()
        for _ in range(runs):
            _ = inference_step(dummy)
        end = time.time()

        avg_ms = (end - start) / runs * 1000
        fps = 1000 / avg_ms
    return avg_ms, fps


total_params = model.count_params()
params_m = total_params / 1e6

flops_g = float('nan')
if get_flops:
    try:
        print("\n--- Running FLOPs calculation (keras-flops) ---")
        
        import contextlib
        import sys
        
        with contextlib.redirect_stdout(None):
             total_flops = get_flops(model, batch_size=input_shape[0])
        
        flops_g = total_flops / 1e9
        print(f"FLOPs calculation completed. Result: {flops_g:.3f} GFLOPs")
        
    except Exception as e:
        print(f"FLOPs calculation failed (keras-flops), reason: {e}")

gpu_time, gpu_fps, cpu_time, cpu_fps = float('nan'), float('nan'), float('nan'), float('nan')

try:
    if tf.config.list_physical_devices('GPU'):
        gpu_time, gpu_fps = measure_inference("/GPU:0", dummy, runs=50)
except Exception as e:
    if str(e) != "Cannot find any device in the device list.":
        print(f"[GPU] Test failed unexpectedly: {e}")

cpu_time, cpu_fps = measure_inference("/CPU:0", dummy, runs=20) 


data = {
    'Model': ['MobileViT_V2_050 + Classifier'],
    'Params (M)': [params_m],
    'FLOPs (G)': [flops_g],
    'GPU Time (ms)': [gpu_time],
    'GPU FPS': [gpu_fps],
    'CPU Time (ms)': [cpu_time],
    'CPU FPS': [cpu_fps]
}
df = pd.DataFrame(data)

df_styled = df.style.format({
    'Params (M)': '{:.3f}',
    'FLOPs (G)': '{:.3f}',
    'GPU Time (ms)': '{:.3f}',
    'GPU FPS': '{:.2f}',
    'CPU Time (ms)': '{:.3f}',
    'CPU FPS': '{:.2f}',
})

print("\n" + "="*80)
print("             🚀 模型性能綜合報告 (Model Performance Summary) 🚀")
print("="*80)
print(df.to_markdown(index=False, floatfmt=".3f"))
print("="*80 + "\n")